In [60]:
import os
import openai
import random
import string

#set up gitpython library - allows us to do GIT calls from python code
from git import Repo
from pathlib import Path #allows us to set up paths and set up directories

import shutil

from bs4 import BeautifulSoup
from PIL import Image

In [40]:
#set up api key - use later
# openai.api_key = os.getenv('OPENAI_API_KEY')

In [41]:
pwd

'C:\\Users\\brian\\OneDrive\\Documents\\GitHub\\briannalytle.github.io'

In [42]:
PATH_TO_BLOG_REPO = Path('C:\\Users\\brian\\OneDrive\\Documents\\GitHub\\briannalytle.github.io\\.git')
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [43]:
PATH_TO_CONTENT.mkdir(exist_ok = True, parents = True)
#using python to make file for us

# Github Pages

1. Create a new public repository on github named **username.github.io**, where username is github username.
2. Clone the repository
3. Create a new file called **index.html**
4. Add some content to **index.html**
5. Push your changes
6. Goto https://username.github.io

## Automatic Updates
Manually adding, commmiting and pushing is tedious.Thus, we will implement a function that handles everyting. Before using openai to generate content for us - we evaluate our code using random strings

In [44]:
def update_blog(commit_message = 'Updated blog'):
    #Gitpython -- repo location
    repo = Repo(PATH_TO_BLOG_REPO)
    #git add .
    repo.git.add(all = True)
    #git commit -m "update blog"
    repo.index.commit(commit_message) 
    #git push
    origin = repo.remote(name = 'origin')
    origin.push()

** Feeding it random text to test update**

In [45]:
random_text_string= 'daftghneianrhwaeklmvc'

In [46]:
#'w' means we are writing to it
with open(PATH_TO_BLOG/"index.html", "w") as f:
    f.write(random_text_string)

In [47]:
update_blog()

Problem: We always overwrite the starting page!
It would be much better to create a new html file and create a link to this file on the homepage. 
Ideally, the routine would automatically assign a file name and make sure that nothing is overwritten!

# pt.2 -
- FUNCTIONS to auto create html page template with placeholders for text and iamges
- use text and images form our api calls

In [70]:
def create_new_blog(title, content, cover_image=Path("../PT Centered Purple.png")):
    
    cover_image = Path(cover_image)
    
    files =len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    if not os.path.exists(path_to_new_content):
        with open(path_to_new_content, "w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exist! Abort")

In [71]:
path_to_new_contet = create_new_blog('Test_title',
                                    'aeuwthrawef', 'apples.jpg')

Blog created


we want an indexed page that has a lsit of all blog posts

In [72]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = BeautifulSoup(index.read())
str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n<title>Document</title>\n</head>\n<body>\n<h1>My BLOG Home Page</h1>\n<a href="index.html">Home</a>\n</body>\n</html>'

In [64]:
#checkign duplicate links

#write blog post  link to index.html file

In [74]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

In [75]:
soup.find_all('a')

[<a href="index.html">Home</a>]

In [76]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = BeautifulSoup(index.read())
        
    links = soup.find_all('a')
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError('Link does already exist!')
        
    link_to_new_blog = BeautifulSoup.new_tag('a', href = Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split('.')[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/'index.html', 'w') as f:
        f.write(str(BeautifulSoup.prettify(formatter = 'html')))

In [77]:
write_to_index(path_to_new_content)


NameError: name 'path_to_new_content' is not defined